In [23]:
import pandas as pd
import string

In [24]:
df = pd.read_csv('Data/validated_top10.csv', index_col=False)
df.head()

,path,accents,sentence
0,common_voice_en_37476269.mp3,United States English,This led to both the Red Terror and the White ...
1,common_voice_en_37740889.mp3,England English,Paper wine labels have long been collected.
2,common_voice_en_37607972.mp3,United States English,The number of chime sets matches the number of...
3,common_voice_en_37437567.mp3,United States English,The Atbara flows only while there is rain in E...
4,common_voice_en_37288403.mp3,United States English,Katsande and her son were accused of voter int...


In [25]:
# loop through each row in the dataframe
for index, row in df.iterrows():
    sent = row['sentence']
    
    sent = sent.translate(str.maketrans('', '', string.punctuation))
    sent = sent.translate(str.maketrans('', '', string.digits))
    sent = ' '.join(sent.split())
    
    x = []
    for c in sent:
        if c in string.printable:
            x.append(c)
    
    df.at[index, 'sentence'] = "".join(x)
    

In [26]:
# save the dataframe to a new tsv file
df.head()

,path,accents,sentence
0,common_voice_en_37476269.mp3,United States English,This led to both the Red Terror and the White ...
1,common_voice_en_37740889.mp3,England English,Paper wine labels have long been collected
2,common_voice_en_37607972.mp3,United States English,The number of chime sets matches the number of...
3,common_voice_en_37437567.mp3,United States English,The Atbara flows only while there is rain in E...
4,common_voice_en_37288403.mp3,United States English,Katsande and her son were accused of voter int...


In [27]:
df.to_csv('Data/validated_top10_cleaned.csv', index=False)